In [4]:
import re

In [7]:
print(re.search(non_terminal_pattern, "TELL"))
print(re.search(pre_terminal_pattern, "Tell"))
print(re.search(terminal_pattern, "chief of staff"))
print(re.search(terminal_pattern, "tell"))

<re.Match object; span=(0, 4), match='TELL'>
<re.Match object; span=(0, 4), match='Tell'>
<re.Match object; span=(0, 14), match='chief of staff'>
<re.Match object; span=(0, 4), match='tell'>


In [116]:
derivation_tree

{'ROOT': {('S', '.'): 1.0,
  ('S', '!'): 1.0,
  ('is', 'it', 'true', 'that', 'S', '?'): 1.0},
 'S': {('NP', 'VP'): 1.0},
 'VP': {('Verb', 'NP'): 1.0},
 'NP': {('Det', 'Noun'): 1.0, ('NP', 'PP'): 1.0},
 'PP': {('Prep', 'NP'): 1.0},
 'Noun': {'president': 1.0,
  'sandwich': 1.0,
  'pickle': 1.0,
  'chief of staff': 1.0,
  'floor': 1.0},
 'Verb': {'ate': 1.0,
  'wanted': 1.0,
  'kissed': 1.0,
  'understood': 1.0,
  'pickled': 1.0},
 'Det': {'the': 1.0, 'a': 1.0, 'every': 1.0},
 'Adj': {'fine': 1.0, 'delicious': 1.0, 'perplexed': 1.0, 'pickled': 1.0},
 'Prep': {'with': 1.0, 'on': 1.0, 'under': 1.0, 'in': 1.0}}

In [128]:
1/6

0.16666666666666666

In [159]:
import re
import random
from collections import ChainMap

non_terminal_pattern = r"^[A-Z]+$"
pre_terminal_pattern = r"^[A-Za-z]{2,}$"
terminal_pattern = r"(^[a-z\s]+)$"

def _spit_probable_choice(items, weights):
    return random.choices(items, weights, k=1)[0]

grammar_text = open("grammar.gr", "rb").read().decode("utf-8")


def _is_non_terminal(text):
    return bool(re.search(non_terminal_pattern, text))

def _is_pre_terminal(text):
    return bool(re.search(pre_terminal_pattern, text))

def _is_terminal(text):
    return bool(re.search(terminal_pattern, text))

def _create_grammar_hash(grammar_text):
    def _is_comment(line):
        if re.search(r"^[#\s()]", line):
            return 1

    valid_symbols = list(filter(lambda text: len(text)>1 and not _is_comment(text), grammar_text.split("\n")))
    cleaned_valid_symbols = list(map(lambda line: line.split("#")[0].strip().split("\t"), valid_symbols))

    non_terminal_hash = {}
    terminal_hash = {}
    for symbol in cleaned_valid_symbols:
        if _is_terminal(symbol[2]):
            if symbol[1] not in terminal_hash:
                terminal_hash[symbol[1]] = {str(symbol[2]): float(symbol[0])}
                continue
            terminal_hash[symbol[1]].update({str(symbol[2]): float(symbol[0])})
            continue
        else:
            print(symbol)
            if symbol[1] not in non_terminal_hash:
                non_terminal_hash[symbol[1]] = {tuple(map(str, symbol[2].split())): float(symbol[0])}
                continue
            non_terminal_hash[symbol[1]].update({tuple(map(str, symbol[2].split())): float(symbol[0])})
    merged = non_terminal_hash.copy()
    merged.update(terminal_hash)
    return terminal_hash, non_terminal_hash, merged

In [167]:
import re
import random
from collections import ChainMap

non_terminal_pattern = r"^[A-Z]+$"
pre_terminal_pattern = r"^[A-Za-z]{2,}$"
terminal_pattern = r"(^[a-z\s]+)$"

def _spit_probable_choice(items, weights):
    return random.choices(items, weights, k=1)[0]

grammar_text = open("grammar.gr", "rb").read().decode("utf-8")


def _is_non_terminal(text):
    return bool(re.search(non_terminal_pattern, text))

def _is_pre_terminal(text):
    return bool(re.search(pre_terminal_pattern, text))

def _is_terminal(text):
    return bool(re.search(terminal_pattern, text))

def _create_grammar_hash(grammar_text):
    def _is_comment(line):
        if re.search(r"^[#\s()]", line):
            return 1

    valid_symbols = list(filter(lambda text: len(text)>1 and not _is_comment(text), grammar_text.split("\n")))
    cleaned_valid_symbols = list(map(lambda line: line.split("#")[0].strip().split("\t"), valid_symbols))

    grammar_hash = {}
    for symbol in cleaned_valid_symbols:
        if _is_terminal(symbol[2]):        
            if symbol[1] not in grammar_hash:
                grammar_hash[symbol[1]] = {str(symbol[2]): float(symbol[0])}
                continue
            grammar_hash[symbol[1]].update({str(symbol[2]): float(symbol[0])})
            continue
        else:
            if symbol[1] not in grammar_hash:
                grammar_hash[symbol[1]] = {tuple(map(str, symbol[2].split())): float(symbol[0])}
                continue
            grammar_hash[symbol[1]].update({tuple(map(str, symbol[2].split())): float(symbol[0])})
            continue
    return grammar_hash

In [168]:
grammar_hash = _create_grammar_hash(grammar_text)

In [162]:
# def _build_sentence(non_terminal_hash, terminal_hash, symbol="ROOT"):
#     items, weights = list(map(lambda text: tuple(map(str, text)), list(non_terminal_hash[symbol].keys()))), list(non_terminal_hash[symbol].values())
#     symbol_choice = _spit_probable_choice(items, weights)
#     for word in symbol_choice:
#         if word in (".", "!"):
#             return word
#         if not _is_terminal(word):
#             if re.search(pre_terminal_pattern, word):
#                 word = _spit_probable_choice(list(terminal_hash[word].keys()), 
#                                              list(terminal_hash[word].values()))
#             else:
#                 word = _spit_probable_choice(list(non_terminal_hash[word].keys()), 
#                                              list(non_terminal_hash[word].values()))
#             return _build_sentence(non_terminal_hash, terminal_hash, word)
#         else:
#             return word

In [ ]:
# v2
# def _build_sentence(non_terminal_hash, terminal_hash, symbol="ROOT"):
#     if symbol in terminal_hash:
#         items = list(terminal_hash[symbol].keys())
#         weights = list(terminal_hash[symbol].values())
#         return _spit_probable_choice(items, weights)

#     if symbol in non_terminal_hash:
#         items = list(non_terminal_hash[symbol].keys())
#         weights = list(non_terminal_hash[symbol].values())
#         rhs = _spit_probable_choice(items, weights)

#         parts = []
#         for sym in rhs:
#             if sym in {".", "!", "?"}:
#                 parts.append(sym)
#             else:
#                 parts.append(_build_sentence(non_terminal_hash, terminal_hash, sym))

#         sentence = " ".join(parts)
#         sentence = re.sub(r"\s+([.!?])", r"\1", sentence)
#         return sentence

#     return symbol

# _build_sentence(non_terminal_hash, terminal_hash)

'every chief of staff wanted every floor in every sandwich in the pickle on every pickle in every president under every president in the floor on a pickle under a pickle in a pickle on a chief of staff with every chief of staff under a president in a pickle on the chief of staff on every president with a floor with a chief of staff with the sandwich on a sandwich in every floor in the chief of staff in every pickle on every sandwich under every chief of staff with the chief of staff on every pickle on every sandwich in a pickle with the chief of staff under the floor under the chief of staff with the president under every chief of staff with every floor on the chief of staff with a president in a sandwich with a pickle on every floor with the sandwich in every sandwich with every chief of staff with a floor in the pickle in every president in the floor with a chief of staff with every floor in the pickle with the chief of staff on the chief of staff on every president with a sandwich o

In [170]:
def _convert_tokens_to_sentence(tokens):
    sentence = " ".join(tokens)
    sentence = re.sub(r"\s+([.!?])", r"\1", sentence)
    sentence = re.sub(r"\s+'", "'", sentence)
    return sentence


def _build_sentence(derivation_tree, symbol="ROOT", _tokens=False):
    if symbol not in derivation_tree:
        return symbol.split() if _tokens else symbol

    if _is_terminal(symbol):
        items = list(derivation_tree[symbol].keys())
        weights = list(derivation_tree[symbol].values())
        return _spit_probable_choice(items, weights)

    if _is_non_terminal(symbol) or _is_pre_terminal(symbol):
        items = list(derivation_tree[symbol].keys())
        weights = list(derivation_tree[symbol].values())
        rhs = _spit_probable_choice(items, weights)

        if isinstance(rhs, tuple):
            tokens = []
            for sym in rhs:
                # tokens.extend(_build_sentence(derivation_tree, sym))
                part = _build_sentence(derivation_tree, sym, _tokens=True)
                if isinstance(part, list):
                    tokens.extend(part)
                else:
                    tokens.extend(part.split())

            if _tokens:
                return tokens
            
            return _convert_tokens_to_sentence(tokens)
        else:
            if _tokens:
                return rhs.split()
            s = rhs
            # s = re.sub(r"\s+([.,!?;:])", r"\1", s)
            # s = re.sub(r"\s+'", "'", s)
            # return s.strip()
            return _convert_tokens_to_sentence([s])

_build_sentence(derivation_tree)

'ROOT'

In [174]:
## V3
def _build_sentence(derivation_tree, symbol="ROOT", _tokens=False):
    if symbol not in derivation_tree:
        return symbol.split() if _tokens else symbol

    ## handling non terminal
    ### being safe
    # choices = derivation_tree[symbol]
    # if not choices:
    #     return [] if _tokens else ""

    ### Logic
    items = list(derivation_tree[symbol].keys())
    weights = list(derivation_tree[symbol].values())
    rhs = _spit_probable_choice(items, weights)

    if isinstance(rhs, tuple):
        tokens = []
        for sym in rhs:
            tokens.extend(_build_sentence(derivation_tree, sym, _tokens=True))
        return tokens if _tokens else _convert_tokens_to_sentence(tokens)
    return rhs.split() if _tokens else rhs.strip()

_build_sentence(grammar_hash)

'is it true that every sandwich under a floor with every president kissed every perplexed pickle?'

| Non Terminal | Symbol | probability |
|--------------|--------|-------------|
| ROOT         | S .    | 0.33        |
| ROOT         | S !    | 0.33        |
| ROOT         | is it true that S ? | 0.33 |
| 

In [ ]:
class Grammar:
    def __init__(self, grammar_file):
        """
        Context-Free Grammar (CFG) Sentence Generator

        Args:
            grammar_file (str): Path to a .gr grammar file
        
        Returns:
            self
        """
        # Parse the input grammar file
        self.rules = None
        self._load_rules_from_file(grammar_file)

    @staticmethod
    def _spit_probable_choice(items, weights):
        return random.choices(items, weights, k=1)[0]

    @staticmethod
    def _convert_tokens_to_sentence(tokens):
        sentence = " ".join(tokens)
        sentence = re.sub(r"\s+([.!?])", r"\1", sentence)
        sentence = re.sub(r"\s+'", "'", sentence)
        return sentence

    def _load_rules_from_file(self, grammar_file):
        """
        Read grammar file and store its rules in self.rules

        Args:
            grammar_file (str): Path to the raw grammar file 
        """
        def _is_comment(line):
            if re.search(r"^[#\s()]", line):
                return 1

        grammar_text = open(grammar_file, "rb").read().decode("utf-8")

        def _is_terminal(text):
            return bool(re.search(terminal_pattern, text))


        valid_symbols = list(filter(lambda text: len(text)>1 and not _is_comment(text), grammar_text.split("\n")))
        cleaned_valid_symbols = list(map(lambda line: line.split("#")[0].strip().split("\t"), valid_symbols))

        grammar_hash = {}
        for symbol in cleaned_valid_symbols:
            if _is_terminal(symbol[2]):        
                if symbol[1] not in grammar_hash:
                    grammar_hash[symbol[1]] = {str(symbol[2]): float(symbol[0])}
                    continue
                grammar_hash[symbol[1]].update({str(symbol[2]): float(symbol[0])})
                continue
            else:
                if symbol[1] not in grammar_hash:
                    grammar_hash[symbol[1]] = {tuple(map(str, symbol[2].split())): float(symbol[0])}
                    continue
                grammar_hash[symbol[1]].update({tuple(map(str, symbol[2].split())): float(symbol[0])})
                continue
        self.rules = grammar_hash

    def sample(self, derivation_tree, max_expansions, start_symbol):
        """
        Sample a random sentence from this grammar

        Args:
            derivation_tree (bool): if true, the returned string will represent 
                the tree (using bracket notation) that records how the sentence 
                was derived
            max_expansions (int): max number of nonterminal expansions we allow

            start_symbol (str): start symbol to generate from

        Returns:
            str: the random sentence or its derivation tree
        """
        
        if start_symbol not in self.rules:
            return start_symbol.strip()
        
        def _tree_expand(symbol, n_expansions=0):
            if (symbol not in self.rules) or (n_expansions >= max_expansions):
                tokens = symbol.split()
                tree = symbol
                return tokens, tree
            
            items = list(self.rules[symbol].keys())
            weights = list(self.rules[symbol].values())
            right_split = self._spit_probable_choice(items=items, weights=weights)
            
            if isinstance(right_split, tuple):
                tokens_list, subtrees = [], []
                for child in right_split:
                    tokens, tree = _tree_expand(child, n_expansions+1)
                    tokens_list.extend(tokens)
                    subtrees.append(tree)
                return tokens_list, f"({symbol} {' '.join(subtrees)})"
            else:
                return right_split.split(), f"({symbol} {right_split})"
        
        tokens, tree_str = _tree_expand(start_symbol)
        if derivation_tree:
            return tree_str
        return self._convert_tokens_to_sentence(tokens)

In [232]:
grammar = Grammar(grammar_file="grammar.gr")

In [233]:
grammar.sample(derivation_tree=False, max_expansions=10, start_symbol="ROOT")

'is it true that the fine pickled president with a chief of staff on a chief of staff on NP PP Prep NP under the chief of staff under Det Noun in a chief of staff pickled a chief of staff on the floor with NP PP Prep NP on Det Noun Prep NP on Det Noun on a sandwich under the delicious sandwich?'